In [1]:
import pandas as pd
import numpy as np
import sys
import keras
from nba_help import get_basic_statistics

Using TensorFlow backend.


In [2]:
from tensorflow.keras.backend import set_session

import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU
config.log_device_placement = True  # to log device placement (on which device the operation ran)
sess = tf.Session(config=config)
set_session(sess)  # set this TensorFlow session as the default session for Keras

In [3]:
score_assist_defend_make_df = pd.read_csv("data/aggregated/assist_new_algo.csv")
score_assist_defend_miss_df = pd.read_csv("data/aggregated/miss_new_algo.csv")

In [4]:
score_assist_defend_make_df = score_assist_defend_make_df.loc[score_assist_defend_make_df['TeamName'].isnull()==False]  
score_assist_defend_miss_df = score_assist_defend_miss_df.loc[score_assist_defend_miss_df['TeamName'].isnull()==False]  

In [5]:
score_assist_defend_make_df = score_assist_defend_make_df.loc[score_assist_defend_make_df['OtherTeamName'].isnull()==False]  
score_assist_defend_miss_df = score_assist_defend_miss_df.loc[score_assist_defend_miss_df['OtherTeamName'].isnull()==False]  

In [6]:
score_assist_defend_make_df['Made'] = 1
score_assist_defend_miss_df['Made'] = 0

In [7]:
score_assist_defend = pd.concat([score_assist_defend_make_df,
                                 score_assist_defend_miss_df],ignore_index=True, sort=False)

In [8]:
def get_train_for_row(row):
    scorer = np.zeros((55, 60))
    assist = np.zeros((55, 60))
    scorer_defend = np.zeros((55, 60))
    assist_defend = np.zeros((55, 60))

    assist_x = int(row['AssistorX'])
    assist_y = int(row['AssistorY'])

    assist[assist_x, assist_y] = 1

    scorer_x = int(row['ScorerX'])
    scorer_y = int(row['ScorerY'])

    scorer[scorer_x, scorer_y] = 1

    pass_defender_x = int(row['PassDefenderX'])
    pass_defender_y = int(row['PassDefenderY'])

    assist_defend[pass_defender_x, pass_defender_y] = 1

    shot_defender_x = int(row['ShotDefenderX'])
    shot_defender_y = int(row['ShotDefenderY'])
    scorer_defend[shot_defender_x, shot_defender_y] = 1

    return (scorer, assist, scorer_defend, assist_defend)

In [9]:
X = []
y = []
for i, row in score_assist_defend.iterrows():
    X.append(get_train_for_row(row))
    y.append(row['Made'])
X = np.array(X).reshape(len(X), 55,60,4)
y = np.array(y)

In [20]:
from keras.models import Sequential
from keras import layers
def get_model():
    model = Sequential()
    model.add(layers.Conv2D(32, (5,5), activation='relu', input_shape=(55, 60, 4), padding='same'))
    model.add(layers.Conv2D(64, (3,3), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D((3, 3)))
    model.add(layers.Conv2D(128, (2,2), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(256, (2,2), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Flatten())
    model.add(layers.Dense(200, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])
    return model

In [21]:
my_callbacks = [
    keras.callbacks.EarlyStopping(patience=2),
    keras.callbacks.ModelCheckpoint(filepath='model.{epoch:02d}-{val_loss:.2f}.h5'),
    keras.callbacks.TensorBoard(log_dir='./logs'),
]

In [22]:
model = get_model()
model.fit(X, y, epochs=100, validation_split=0.2, shuffle=True, callbacks=my_callbacks)

Train on 17630 samples, validate on 4408 samples
Epoch 1/100
17630/17630 [==============================] - 10s 556us/step - loss: 0.6694 - acc: 0.6161 - val_loss: 0.9406 - val_acc: 0.0000e+00
Epoch 2/100
17630/17630 [==============================] - 10s 554us/step - loss: 0.6646 - acc: 0.6189 - val_loss: 0.9733 - val_acc: 0.0000e+00
Epoch 3/100
17630/17630 [==============================] - 10s 545us/step - loss: 0.6646 - acc: 0.6189 - val_loss: 0.9676 - val_acc: 0.0000e+00


In [ ]:
url = 'http://stats.nba.com/stats/shotchart'
headers = 